## Machine Learning Pipeline の構築

[Azure Machie Learning Pipeline](https://docs.microsoft.com/ja-JP/azure/machine-learning/concept-ml-pipelines) は再利用可能な機械学習パイプラインを実現します。モデル学習やバッチ推論で利用できます。構築した Machine Learning Pipeline は、[Github Actions](https://github.com/Azure/aml-run) や [Azure Data Factory](https://docs.microsoft.com/ja-JP/azure/data-factory/transform-data-machine-learning-service) などのサービスから呼び出すことができます。

本 Noteobok では [3-aml-run.ipynb](./3-aml-run.ipynb) の処理をパイプラインとして再構築します。

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Dataset,Environment
from azureml.widgets import RunDetails

### Machine Learning Pipeline 関連ライブラリ

In [ ]:
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
from azureml.core.compute import AmlCompute
aml_compute = AmlCompute(ws, "gpucluster")

### 実行構成

In [ ]:
from azureml.core.runconfig import RunConfiguration
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
env = Environment.get(ws, "pytorch-env")
run_config.environment = env

### データセット

In [ ]:
dataset = Dataset.get_by_name(ws, name='cifar10')
ds_input = dataset.as_named_input('input1')

In [ ]:
step1 = PythonScriptStep(name="train_step",
                         script_name="train.py",
                         arguments=[
                            '--data_path', ds_input.as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92],
                         #inputs=[ds_input],
                         compute_target=aml_compute, 
                         source_directory='./code/pytorch-cloud',
                         runconfig=run_config,
                         allow_reuse=True)
print("step1 作成完了")

In [ ]:
steps = [step1]

In [ ]:
pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline 構築完了")

In [ ]:
pipeline1.validate()
print ("Pipeline 検証完了")

In [ ]:
run = pipeline1.submit("5-aml-pipeline-run")

In [ ]:
run.wait_for_completion()

### Endpoint として公開

In [ ]:
pipeline1.publish("5-aml-pipeline")